# Pipeline Examples

This notebook demonstrates the end-to-end process of building a machine learning pipeline using PLAID datasets and PLAID’s scikit-learn-compatible blocks.

### 📦 Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore', module='sklearn')
warnings.filterwarnings("ignore", message=".*IProgress not found.*")

import os
from pathlib import Path

import yaml
import numpy as np
import optuna

from datasets.utils.logging import disable_progress_bar
from datasets import load_dataset

from sklearn.base import clone
from sklearn.pipeline import Pipeline

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from sklearn.multioutput import MultiOutputRegressor

from sklearn.model_selection import KFold, GridSearchCV

from plaid.bridges.huggingface_bridge import huggingface_dataset_to_plaid, huggingface_description_to_problem_definition
from plaid.pipelines.sklearn_block_wrappers import WrappedPlaidSklearnTransformer, WrappedPlaidSklearnRegressor
from plaid.pipelines.plaid_blocks import PlaidTransformedTargetRegressor, PlaidColumnTransformer

disable_progress_bar()
n_processes = min(max(1, os.cpu_count()), 6)

## MMGP for `U1` field prediction of `Tensile2d` dataset

Key steps covered:

- **Loading and preparing the PLAID dataset** using Hugging Face integration and PLAID’s dataset classes  
- **Standardizing features** with PLAID-wrapped scikit-learn transformers for scalars and fields  
- **Dimensionality reduction** of flow fields via Principal Component Analysis (PCA) to reduce output complexity  
- **Regression modeling** of PCA coefficients from scalar inputs using Gaussian Process regression  
- **Pipeline assembly** combining transformations and regressors into a single scikit-learn-compatible workflow  
- **Hyperparameter tuning** using Optuna and scikit-learn’s `GridSearchCV`
- **Model evaluation** using cross-validation and appropriate metrics  
- **Best practices** for working with PLAID datasets and pipelines in a reproducible and modular manner

### 📥 Load Dataset

We load the `Tensile2d` dataset from Hugging Face and restrict ourselves to the first 24 samples of the training set.

In [ ]:
hf_dataset = load_dataset("PLAID-datasets/Tensile2d", split="all_samples[:24]")
dataset_train, _ = huggingface_dataset_to_plaid(hf_dataset, processes_number = 6, verbose = False)

In [ ]:
print("dataset_train =", dataset_train)

In [ ]:
try:
    filename = Path(__file__).parent.parent.parent / "docs" / "source" / "notebooks" / "config_pipeline2.yml"
except NameError:
    filename = "config_pipeline2.yml"

with open(filename, 'r') as f:
    config = yaml.safe_load(f)

all_feature_id = config['input_scalar_scaler']['in_features_identifiers'] +\
    config['pca_nodes']['in_features_identifiers'] + config['pca_u1']['in_features_identifiers']

In [ ]:
dataset_train = dataset_train.from_features_identifier(all_feature_id)
print("dataset_train:", dataset_train)
print("scalar names =", dataset_train.get_scalar_names())
print("field names =", dataset_train.get_field_names())

In [ ]:
from MM_node import MeshMorphingInterpolationTransformer
# mm = MeshMorphingInterpolationTransformer(common_mesh_id = 0)

In [ ]:
# mm.fit(dataset_train)

In [ ]:
# transformed_dataset = mm.transform(dataset_train)


In [ ]:
# transformed_dataset[0].get_field("U1")


In [ ]:
# from Muscat.Bridges.CGNSBridge import CGNSToMesh

# print(transformed_dataset)
# index = 1
# print(transformed_dataset[index].get_all_mesh_times())
# print(transformed_dataset[index].get_field("coords_X", time = 0.).shape)
# print(transformed_dataset.extra_data[index].get_field("coords_X").shape)
# print(transformed_dataset[index].get_field("U1", time = 0.).shape)
# print(transformed_dataset.extra_data[index].get_field("U1"))
# mesh = CGNSToMesh(transformed_dataset[index].get_mesh(0.))
# mesh_o = CGNSToMesh(transformed_dataset.extra_data[index].get_mesh())
# print(mesh)
# print(mesh_o)
# print(">>", transformed_dataset[0].get_field("U1", time = 0.).shape)
# print(">>", transformed_dataset[1].get_field("U1", time = 0.).shape)
# print(">>", transformed_dataset[0].get_nodes().shape)
# print(">>", transformed_dataset[1].get_nodes().shape)


In [ ]:
# print(CGNSToMesh(dataset_train[0].get_mesh(0.)))
# print(CGNSToMesh(dataset_train[1].get_mesh(0.)))

In [ ]:
# inv_transformed_dataset = mm.inverse_transform(transformed_dataset)

In [ ]:
# index = 0
# print(inv_transformed_dataset[index].get_scalar('P'))
# print(dataset_train[index].get_scalar('P'))
# print("==")
# print(np.linalg.norm(inv_transformed_dataset[index].get_nodes()-dataset_train[index].get_nodes()))
# print("==")
# print(inv_transformed_dataset[index].get_field("U1"))
# print(dataset_train[index].get_field("U1"))
# print(np.linalg.norm(inv_transformed_dataset[index].get_field("U1")-dataset_train[index].get_field("U1")))


In [ ]:
mm_preprocessor = MeshMorphingInterpolationTransformer(common_mesh_id = 12)

preprocessor = PlaidColumnTransformer(
    [
        ('input_scalar_scaler', WrappedPlaidSklearnTransformer(MinMaxScaler(), **config['input_scalar_scaler'])),
        ('pca_nodes', WrappedPlaidSklearnTransformer(PCA(n_components=24), **config['pca_nodes'])),
    ]
)

preprocessor

In [ ]:
postprocessor = WrappedPlaidSklearnTransformer(PCA(n_components=24), **config['pca_u1'])

kernel = Matern(length_scale_bounds=(1e-8, 1e8), nu = 2.5)

gpr = GaussianProcessRegressor(
    kernel=kernel,
    optimizer='fmin_l_bfgs_b',
    n_restarts_optimizer=1,
    random_state=42)

reg = MultiOutputRegressor(gpr)

def length_scale_init(X):
    return np.ones(X.shape[1])

dynamics_params_factory = {'estimator__kernel__length_scale':length_scale_init}

regressor = WrappedPlaidSklearnRegressor(reg, **config['regressor_mach'], dynamics_params_factory = dynamics_params_factory)

target_regressor = PlaidTransformedTargetRegressor(
    regressor=regressor,
    transformer=postprocessor
)
target_regressor

In [ ]:
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("regressor", target_regressor),
    ]
)
pipeline

In [ ]:
dataset_train_1 = mm_preprocessor.fit_transform(dataset_train)

In [ ]:
pipeline.fit(dataset_train_1)

In [ ]:
dataset_pred_1 = pipeline.predict(dataset_train_1)

In [ ]:
for index in range(24):
    print("rel_dif =", np.linalg.norm(dataset_pred_1[index].get_field("U1") - dataset_train_1[index].get_field("U1")))

In [ ]:
dataset_pred = mm_preprocessor.inverse_transform(dataset_pred_1)

In [ ]:
for index in range(24):
    print("rel_dif =", np.linalg.norm(dataset_pred[index].get_field("U1") - dataset_train[index].get_field("U1")))

In [ ]:
print(dataset_pred[0].get_field_names())

In [ ]:
print(dataset_train[0].get_field_names())